In [27]:
import pandas as pd
from neo4j import GraphDatabase
import csv
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import pairwise_distances
import ast


In [65]:
df = pd.read_csv("final_movie.csv")

df.head(5)

df = df[(df['genres'].str.strip() != "[]") & (df['Director'].str.strip() != "[]")]


In [67]:
# Sampling data
df_movies = df.copy()
sub_df = df_movies.sample(n=1000, random_state=42)


In [70]:
# get unique genres
genres = []
sub_df['genres'].apply(lambda row: genres.extend(ast.literal_eval(row)))

# get unique directors
directors = []
sub_df['Director'].apply(lambda row: directors.extend(ast.literal_eval(row)))


genres = set(genres)
directors = set(directors)

## Setting up graph database

In [10]:
uri = "neo4j://localhost:7687"
user = "neo4j"
password = "vampire-float-olivia-maestro-sleep-4222" 

#### Make sure you can run this on your browser - initiate a Neo4j server ####

In [11]:
driver = GraphDatabase.driver(uri, auth=(user, password))

In [43]:
# kill switch
def delete_all_data(tx):
    tx.run("MATCH (n) DETACH DELETE n")

with driver.session() as session:
    session.write_transaction(delete_all_data)

driver.close()

/var/folders/5w/zg23p5bd3bnc86d75_pkn99m0000gn/T/ipykernel_98724/3803800313.py:6: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(delete_all_data)


### Movie node

In [44]:
def create_movie_node(tx, row):
    tx.run("""
    CREATE (:movie {
            title: $title,
            overview: $overview,
            vote_average: $vote_average, 
            vote_count: $vote_count

    })
    """, title=row['title'], overview=row['overview'], vote_average=row['vote_average'],
                vote_count=row['vote_count'])
    

with driver.session() as session:
    for index, row in sub_df.iterrows():
        session.write_transaction(create_movie_node, row)

    

/var/folders/5w/zg23p5bd3bnc86d75_pkn99m0000gn/T/ipykernel_98724/556782445.py:16: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_movie_node, row)


### Director Node

In [ ]:
def create_director_node(tx, row):
    pass